In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import argparse
import json
import pandas as pd
import keras
import tensorflow as tf
assert(tf.__version__ == "1.15.2")
assert(keras.__version__ == "2.2.4")
import warnings
warnings.filterwarnings('ignore')

# Define working directory
ROOT_DIR = os.path.abspath("../")
sys.path.append(ROOT_DIR)
ROOT_DIR = os.path.abspath("../../")
sys.path.append(ROOT_DIR)

from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from utils.dataset import BloodCellDataset, BloodCellDatasetFromDataFrame
from utils.configs import BloodCellConfig_A, BloodCellConfig_B

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "maskrcnn-master","logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "maskrcnn-master", "mask_rcnn_coco.h5")

# Directory of images to run detection on
IMAGE_DIR_A = os.path.join(ROOT_DIR, "BCCD_Dataset/BCCD")

# Directory of images to run detection on
IMAGE_DIR_B = os.path.join(ROOT_DIR, "segmentation_WBC")

if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

IMAGE_COLLECTION_PATH = r'../blood_cell_dataset.pkl'


Using TensorFlow backend.


In [3]:
if not os.path.exists(IMAGE_COLLECTION_PATH):
    raise Exception(f"IC_PATH @ {IMAGE_COLLECTION_PATH} not found.")
else:
    dataset = pd.read_pickle(IMAGE_COLLECTION_PATH)
    train_samples_A = dataset[(dataset["role"] == "train")&(dataset["origin"] == "A")]
    train_samples_B = dataset[(dataset["role"] == "train")&(dataset["origin"] == "B")]
    val_samples = dataset[dataset["role"] == "val"]

print(f"Number of samples in training set A: {len(train_samples_A)}")
print(f"Number of samples in training set B: {len(train_samples_B)}")
print(f"Number of samples in valid set: {len(val_samples)}")

Number of samples in training set A: 295
Number of samples in training set B: 100
Number of samples in valid set: 69


In [4]:
dataset_train_A = BloodCellDatasetFromDataFrame(train_samples_A, directory_path_A = IMAGE_DIR_A)
dataset_train_A.load_kernel()

dataset_train_B = BloodCellDatasetFromDataFrame(train_samples_B, directory_path_B = IMAGE_DIR_B)
dataset_train_B.load_kernel()

dataset_val = BloodCellDatasetFromDataFrame(val_samples, directory_path_A = IMAGE_DIR_A)
dataset_val.load_kernel()

In [5]:
config_A = BloodCellConfig_A()
config_A.STEPS_PER_EPOCH = 400
config_A.LEARNING_RATE = 1e-3
config_A.NAME = "BloodCell"

config_B = BloodCellConfig_B()
config_B.STEPS_PER_EPOCH = 100
config_B.LEARNING_RATE = 1e-4
config_B.NAME = "BloodCell"

# Stage A: train model on train_A (BBox and Classes)

In [ ]:
# Create model in training mode
model = modellib.MaskRCNN(mode="training", config=config_A,
                          model_dir=MODEL_DIR)
model.load_weights(COCO_MODEL_PATH, by_name=True,
                   exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", "mrcnn_bbox", "mrcnn_mask"])

In [ ]:
model.train(dataset_train_A, dataset_val,
            learning_rate=config_A.LEARNING_RATE,
            epochs=5,
            layers='heads')

In [ ]:
model.train(dataset_train_A, dataset_val,
            learning_rate=config_A.LEARNING_RATE,
            epochs=25,
            layers="all")

In [6]:
model_branch = "bloodcell20201215T1523"
checkpoint_epoch = 25

model = modellib.MaskRCNN(mode="training", config=config_B,
                          model_dir=MODEL_DIR)
model_path = os.path.join(MODEL_DIR, model_branch,
                              f"mask_rcnn_bloodcell_{checkpoint_epoch:04d}.h5")
# Load the last model you trained and continue training
model.load_weights(model_path, by_name=True)
model.train(dataset_train_B, dataset_val,
            learning_rate=config_B.LEARNING_RATE,
            epochs=40,
            layers="mrcnn_mask_heads")







Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
box_ind is deprecated, use box_indices instead






Re-starting from epoch 25

Starting at epoch 25. LR=0.0001

Checkpoint Path: /home/jiayu.gan/Inarix/HEC/maskrcnn/maskrcnn-master/logs/bloodcell20201215T1523/mask_rcnn_bloodcell_{epoch:04d}.h5
Selecting layers to train
In model:  rpn_model
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_mask_conv4       (TimeDistributed)
mrcnn_mask_bn4         (TimeDistributed)
mrcnn_mask_deconv      (TimeDistributed)

Multiprocessing? True, number of workers 2




Epoch 26/40
100/100 [==============================] - 39s 387ms/step - loss: 0.5626 - rpn_class_loss: 0.0000e+00 - rpn_bbox_loss: 0.0000e+00 - mrcnn_class_loss: 

100/100 [==============================] - 30s 303ms/step - loss: 0.1228 - rpn_class_loss: 0.0000e+00 - rpn_bbox_loss: 0.0000e+00 - mrcnn_class_loss: 0.0000e+00 - mrcnn_bbox_loss: 0.0000e+00 - mrcnn_mask_loss: 0.1228 - val_loss: 2.1406 - val_rpn_class_loss: 0.0000e+00 - val_rpn_bbox_loss: 0.0000e+00 - val_mrcnn_class_loss: 0.0000e+00 - val_mrcnn_bbox_loss: 0.0000e+00 - val_mrcnn_mask_loss: 2.1406
Epoch 33/40
100/100 [==============================] - 31s 309ms/step - loss: 0.1169 - rpn_class_loss: 0.0000e+00 - rpn_bbox_loss: 0.0000e+00 - mrcnn_class_loss: 0.0000e+00 - mrcnn_bbox_loss: 0.0000e+00 - mrcnn_mask_loss: 0.1169 - val_loss: 2.4679 - val_rpn_class_loss: 0.0000e+00 - val_rpn_bbox_loss: 0.0000e+00 - val_mrcnn_class_loss: 0.0000e+00 - val_mrcnn_bbox_loss: 0.0000e+00 - val_mrcnn_mask_loss: 2.4679
Epoch 34/40
100/100 [==============================] - 31s 306ms/step - loss: 0.1091 - rpn_class_loss: 0.0000e+00 - rpn_bbox_loss: 0.0000e+00 - mrcnn_class_loss: 0.0000e+00 - mrcnn_bbox_lo